In [1]:
import pandas as pd
import numpy as np

In [2]:
pop = pd.read_csv('./Population-EstimatesData.csv')
obs = pd.read_csv('./obesity_clean.csv', index_col = 0)

In [3]:
pop.shape

(45325, 96)

In [4]:
obs.shape

(8022, 6)

In [5]:
pop.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030', '2031',
       '2032', '2033', '2034', '2035', '2036', '2037', '2038', '2039', '2040',
       '2041', '2042', '2043', '2044', '2045', '2046', '2047', '2048', '2049',
       '2050', 'Unnamed: 95'],
      dtype='object')

In [6]:
pop_clean = pd.DataFrame()

In [7]:
# We only need data from 1975-2016 and a few other columns, copying from original df
for column in pop.columns:
    if column in ['Country Name', 'Country Code', 'Indicator Code']:
        pop_clean[column] = pop[column]

In [8]:
pop_clean.head(3)

,Country Name,Country Code,Indicator Code
0,Arab World,ARB,SP.POP.DPND
1,Arab World,ARB,SP.POP.DPND.OL
2,Arab World,ARB,SP.POP.DPND.YG


In [9]:
# copy the years we have in common between obs and pop to pop_clean
for year in range(1965, 2017):
    if str(year) in pop.columns:
        pop_clean[year] = pop[str(year)]

In [10]:
pop_clean.head(3)

,Country Name,Country Code,Indicator Code,1965,1966,1967,1968,1969,1970,1971,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Arab World,ARB,SP.POP.DPND,93.201290,94.087535,94.485922,94.544610,94.434251,94.226666,94.523203,...,65.784310,64.778501,63.913830,63.201624,62.572328,62.089329,61.730765,61.446530,61.195307,61.283615
1,Arab World,ARB,SP.POP.DPND.OL,6.947790,7.015592,7.053468,7.070116,7.075065,7.073558,7.118325,...,6.759558,6.711798,6.676818,6.661625,6.668728,6.686883,6.720703,6.776496,6.858187,6.981482
2,Arab World,ARB,SP.POP.DPND.YG,85.872972,86.674248,87.052932,87.131942,87.045298,86.848912,87.099372,...,57.203135,56.144645,55.247367,54.517731,53.919135,53.468340,53.136254,52.861795,52.595712,52.611349


In [11]:
# SP.POP.TOTL minus SP.POP.0014.TO should give us close to the adult populations
pop_clean = pop_clean[(pop_clean['Indicator Code'] == 'SP.POP.TOTL') | (pop_clean['Indicator Code'] == 'SP.POP.0014.TO')]

In [12]:
pop_clean.head(2)

,Country Name,Country Code,Indicator Code,1965,1966,1967,1968,1969,1970,1971,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
84,Arab World,ARB,SP.POP.0014.TO,47089854.0,48668380.0,50184681.0,51657737.0,53108292.0,54542164.0,56131706.0,...,115231484.0,116661771.0,118267397.0,120038945.0,121934515.0,124007632.0,126213709.0,128449757.0,130629538.0,133190635.0
166,Arab World,ARB,SP.POP.TOTL,105736431.0,108758610.0,111899364.0,115136178.0,118437195.0,121785650.0,125164745.0,...,330290716.0,338395961.0,346629220.0,354890042.0,363158703.0,371443547.0,379705719.0,387907748.0,396028278.0,404024433.0


In [13]:
# Lets match the columns in obs with with columns in pop_clean
pop_clean.rename(columns = {'Country Code' : 'country_code', 'Country Name' : 'country'}, inplace = True)

In [14]:
# Transform Dataframe to build a new column
pop_clean = pop_clean.\
                pivot_table(columns = ['country', 'country_code', 'Indicator Code']).\
                unstack(level = -1).\
                reset_index().\
                rename_axis('', axis = 'columns').\
                rename(columns = {'level_0' : 'year'}).\
                sort_values(by = ['country', 'year'])

In [15]:
# Resetting index again for more logical order
pop_clean.reset_index(drop = True, inplace = True)

In [16]:
# Move column order for readbility
pop_clean = pop_clean[['country', 'country_code', 'year', 'SP.POP.TOTL', 'SP.POP.0014.TO']]

In [17]:
pop_clean.head(3)

,country,country_code,year,SP.POP.TOTL,SP.POP.0014.TO
0,Afghanistan,AFG,1965,9956320.0,4279406.0
1,Afghanistan,AFG,1966,10174836.0,4408352.0
2,Afghanistan,AFG,1967,10399926.0,4537722.0


In [18]:
pop_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13416 entries, 0 to 13415
Data columns (total 5 columns):
country           13416 non-null object
country_code      13416 non-null object
year              13416 non-null int64
SP.POP.TOTL       13325 non-null float64
SP.POP.0014.TO    12162 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 524.1+ KB


In [19]:
# The NA's in the child population can be filled with zeroes and the total population used
pop_clean['SP.POP.0014.TO'].fillna(0, inplace = True)

In [20]:
# NA's in the total population have to be removed or errors will occur, less than 100 total
pop_clean = pop_clean.drop(pop_clean[pop_clean['SP.POP.TOTL'].isnull()].index)

In [21]:
pop_clean['adult_pop'] = pop_clean['SP.POP.TOTL'] - pop_clean['SP.POP.0014.TO']

In [22]:
pop_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13325 entries, 0 to 13415
Data columns (total 6 columns):
country           13325 non-null object
country_code      13325 non-null object
year              13325 non-null int64
SP.POP.TOTL       13325 non-null float64
SP.POP.0014.TO    13325 non-null float64
adult_pop         13325 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 728.7+ KB


In [23]:
pop_clean.describe()

,year,SP.POP.TOTL,SP.POP.0014.TO,adult_pop
count,13325.000000,1.332500e+04,1.332500e+04,1.332500e+04
mean,1990.564803,1.789865e+08,5.754216e+07,1.214443e+08
std,15.008812,6.117475e+08,1.946778e+08,4.221257e+08
min,1965.000000,4.468000e+03,0.000000e+00,4.468000e+03
25%,1978.000000,8.780420e+05,2.853040e+05,5.360830e+05
50%,1991.000000,6.249165e+06,2.007206e+06,3.953767e+06
75%,2004.000000,3.688454e+07,1.119047e+07,2.391570e+07
max,2016.000000,7.426103e+09,1.935527e+09,5.490576e+09


In [24]:
pop_clean.describe(include = 'O')

,country,country_code
count,13325,13325
unique,258,258
top,Least developed countries: UN classification,CHE
freq,52,52


In [25]:
# Lastly, let's get rid of nations not in both datasets
pop_clean = pop_clean[pop_clean['country_code'].isin(obs['country_code']) == True]

In [26]:
pop_clean.head(5)

,country,country_code,year,SP.POP.TOTL,SP.POP.0014.TO,adult_pop
0,Afghanistan,AFG,1965,9956320.0,4279406.0,5676914.0
1,Afghanistan,AFG,1966,10174836.0,4408352.0,5766484.0
2,Afghanistan,AFG,1967,10399926.0,4537722.0,5862204.0
3,Afghanistan,AFG,1968,10637063.0,4670217.0,5966846.0
4,Afghanistan,AFG,1969,10893776.0,4808162.0,6085614.0


In [27]:
pop_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9795 entries, 0 to 13415
Data columns (total 6 columns):
country           9795 non-null object
country_code      9795 non-null object
year              9795 non-null int64
SP.POP.TOTL       9795 non-null float64
SP.POP.0014.TO    9795 non-null float64
adult_pop         9795 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 535.7+ KB


In [28]:
# All clean, let's save this thing
pop_clean.to_csv('./pop_clean.csv')